## Description
Many earth observation datasets are available at regular intervals over long periods of time. This enables us to detect changes on the Earth’s surface. In this session, you will learn how to apply change detection technique in remote sensing used to create NDVI change detection maps. 

## Aims of the practical session
* Create ROI and load it
* Collect images 
* Calculate NDVI for before and after Image
* Comparing two classified images to see the changes of NDVI during the time

## Getting started

### Load packages

Import GEE packages that are needed for the analysis.

In [1]:
import ee
import geemap
# ee.Authenticate()

### Connect to Google Earth Engine (GEE)

Connect to the GEE to have access computing tools and GEE datasets.
You may be required to input your Google account for authorization.

In [2]:
Map = geemap.Map()
# Map.add_basemap('HYBRID')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### Adding Region of Interest (ROI)

Create ROI that we want to work on it and then add and display it on the GEE map.
Import the downloaded shapefile for Canberra central from your computer path as ROI.

In [3]:
# # # load shapefile (ROI)
shp_path = 'D:/GEE Tutorials/NOV21_ACT_LOC_POLYGON_shp/Canberra_central.shp'
geometry = geemap.shp_to_ee(shp_path)
Map.addLayer(geometry, {}, 'Canberra central SHP')
Map.centerObject(geometry);

C:\Users\Abolfazl\Anaconda3\envs\gee\lib\site-packages\geopandas\io\file.py:299: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


### Post Image collection 
An ImageCollection is a stack or sequence of images. An ImageCollection can be loaded by pasting an Earth Engine asset ID into the ImageCollection constructor. You can find ImageCollection IDs in the <a href="https://developers.google.com/earth-engine/datasets">data catalog</a>. 

We will:
* Load post Sentinel-2 images for the anlysis
* Filter a collection by date range
* Select bands
* Clip based on geometry
* Visualize Image

In [4]:
#Import Post-Event Imagery
imagePo = ee.ImageCollection('COPERNICUS/S2') \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
  .filterDate('2021-01-01', '2021-12-31') \
  .filterBounds(geometry) \
  .select('B.*')

print(imagePo.size().getInfo());

49


In [5]:
# # # Visualize Image
vis_params = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}

Map.addLayer(imagePo.median().clip(geometry), vis_params, 'Sent-2 After Image' );
Map

Map(bottom=158764.0, center=[-35.17380831799957, 149.06250000000003], controls=(WidgetControl(options=['positi…

### Calculate NDVI time-series for post image
The purpose of this part is to calculate time series NDVI on remotely sensed data. The NDVI is a vegetation index that describes the difference between visible and near-infrared reflectance of vegetation cover and can be used to estimate the density of green on an area of land.

We will:
* Define a function to calculate NDVI
* Perform time series of the study area based on post imagery
* Clip based on geometry
* Map the results

### Create post-NDVI map

In [6]:
#Create Post-NDVI Map

def func_ndvipo(img):
  index = img.normalizedDifference(['B8', 'B4'])
  clip = index.clip(geometry);#for more regions is not necessary
  return clip.rename('Ndvi').copyProperties(img,['system:time_start','system:time_end'])

ndviPo = imagePo.map(func_ndvipo);

In [7]:
# mean value composite for Before image(calculate mean value of ndvi in every pixel)
mvcPo = ndviPo.mean();

In [8]:
Map.addLayer(mvcPo, {min: -1, max: 1, 'palette': ['white', 'red']}, 'NDVI After Event');
Map

Map(bottom=158764.0, center=[-35.17380831799957, 149.06250000000003], controls=(WidgetControl(options=['positi…

In [9]:
#print(mvcPo.getInfo())

### Create chart for post image
We can use ipygee to generate a chart of the NDVI variation in the analysis period for any point in the ROI:

In [10]:
from ipygee import chart

ndviPo = chart.Image.series(**{
    'imageCollection': ndviPo,
    'region': geometry,
    'reducer': ee.Reducer.mean(),
    'scale': 10,
    'xProperty': 'system:time_start'})
ndviPo.renderWidget(width='100%')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [11]:
# # # create chart for post image
# chartPo = ui.Chart.image.series(
#   ndviPo,table, ee.Reducer.median(),10,'system:time_start'
# ).setChartType('ScatterChart') \
# .setOptions({
#   'title' : 'Ndvi changes over a year',
#    'vAxis':  {title: 'Ndvi values'},
#    'hAxis':  {title: 'Date'},
#   'lineWidth' : 1,
#   'pointSize' : 2,
#   'series' : {
#      '0':  {color: 'red'}
#   }
# })

# print(chartPo)

### Pre Image collection 

We will:
* Load pre Sentinel-2 images for the anlysis
* Filter a collection by date range
* Select bands
* Clip based on geometry
* Visualize Image

In [12]:
#Import Pre-Event Imagery
imagePre = ee.ImageCollection('COPERNICUS/S2') \
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
  .filterDate('2017-01-01', '2017-12-31') \
  .filterBounds(geometry) \
  .select('B.*')

print(imagePre.size().getInfo())

34


In [13]:
# # # Visualize Image
vis_params = {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}

Map.addLayer(imagePre.median().clip(geometry), vis_params, 'Sent-2 Before Image' );
Map

Map(bottom=158764.0, center=[-35.17380831799957, 149.06250000000003], controls=(WidgetControl(options=['positi…

### Calculate NDVI time-series for pre image

We will:
* Define a function to calculate NDVI
* Perform time series of the study area based on pre imagery
* Clip based on geometry
* Map the results

In [14]:
#Create pre-NDVI Map

def func_ndvipre(img):
  index = img.normalizedDifference(['B8', 'B4'])
  clip = index.clip(geometry);#for more regions is not necessary
  return clip.rename('Ndvi').copyProperties(img,['system:time_start','system:time_end'])

ndviPre = imagePre.map(func_ndvipre)

In [15]:
# mean value composite for Before image(calculate mean value of ndvi in every pixel)
mvcPre = ndviPre.mean();

In [16]:
Map.addLayer(mvcPre, {min: -1, max: 1, 'palette': ['white', 'red']}, 'NDVI Before Event');
Map

Map(bottom=158764.0, center=[-35.17380831799957, 149.06250000000003], controls=(WidgetControl(options=['positi…

### Create chart for pre image
We can use ipygee to generate a chart of the NDVI variation in the analysis period for any point in the ROI:

In [17]:
from ipygee import chart

ndviPre = chart.Image.series(**{
    'imageCollection': ndviPre,
    'region': geometry,
    'reducer': ee.Reducer.mean(),
    'scale': 10,
    'xProperty': 'system:time_start'})
ndviPre.renderWidget(width='100%')

HTML(value='<embed src=data:image/svg+xml;charset=utf-8;base64,PD94bWwgdmVyc2lvbj0nMS4wJyBlbmNvZGluZz0ndXRmLTg…

In [18]:
# # # 
# chartPre = ui.Chart.image.series(
#   ndviPre,table, ee.Reducer.median(),10,'system:time_start'
# ).setChartType('ScatterChart') \
# .setOptions({
#   'title' : 'Ndvi changes over a year',
#    'vAxis':  {title: 'Ndvi values'},
#    'hAxis':  {title: 'Date'},
#   'lineWidth' : 1,
#   'pointSize' : 2,
#   'series' : {
#      '0':  {color: 'red'}
#   }
# })

# print(chartPre)

### <a name="ex3"></a> Challenge

<div class="alert alert-block alert-danger">

- Try to improve the NDVI plots by setting options like changing xlabel/ylabel, select good title, change font/font size etc.
    
    
</div>    

### Create NDVI Difference Map
Here, we apply codes to create NDVI difference map based on pre and post results.
We will: 
* Create NDVI Difference Map
* show differences
* Map diff results

In [19]:
# # # Create NDVI Difference Map
diff = mvcPo.subtract(mvcPre);
#print(diff.getInfo())

In [20]:
Map.addLayer(diff,
            {min: -2, max: 2, 'palette': ['white', 'red']},
            'NDVI Difference');
Map

Map(bottom=158764.0, center=[-35.17380831799957, 149.06250000000003], controls=(WidgetControl(options=['positi…

### Export the result
Export the result directly to your computer

In [21]:
# # # Export the result
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'NDVI_Diff.tif')

In [22]:
diffClip = diff.clipToBoundsAndScale(**{
  'geometry': geometry,
  'scale': 10,  # meters
})

In [114]:
geemap.ee_export_image(diffClip, filename=out_file)

Generating URL ...
Please wait ...
Data downloaded to C:\Users\Abolfazl\Downloads\NDVI_Diff.tif


<span style='background:yellow'> <span style="font-size:16.0pt"> Exercise  </span>

### Exercise 1 - NDVI change detection on other datasets
Try to use image collection to load Landsat-8 images and calculate NDVI for pre and post-events, and show the differences. 

#### Then try to check the following exercises:
* What are the differences between NDVI for Sentinel-2 and Landsat-8?
* Is the results of NDVI varied?
* How is the change detection map based on NDVI of Landsat-8?
* Show the differences between two different data. 
* How spatially different the change detection maps of two different data?

## References
This is where the references go. For exmaple:

* Wu, Q., (2020). geemap: A Python package for interactive mapping with Google Earth Engine. The Journal of Open Source Software, 5(51), 2305. https://doi.org/10.21105/joss.02305
* "Earth Observation: Data, Processing and Applications" book. Available through Wattle, or http://www.crcsi.com.au/earth-observation-series.

## Additional information

**License:** The code in this notebook was initially created by the team at [Digital Earth Australia](https://github.com/GeoscienceAustralia/dea-notebooks), and has been modified by Abolfazl Abdollahi. The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 

**Contact:** If you need assistance, please post a question on the ENGN3903 Wattle site.

**Last modified:** June 2022

### Exercise answers

<a name="ex1answer">Answer to Exercise 1</a>

In [27]:
# # # Make a cloud-free Landsat 8 TOA composite (Before imagery).
# landsatPre = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
#     .filterDate('2018-01-01', '2018-12-31')\
#     .filterBounds(geometry) \



# vis_params = {'min': 0, 'max': 3000, 'bands': ['B5', 'B4', 'B3']}

# # Map.centerObject(point, 8)
# Map.addLayer(landsatPre.median().clip(geometry), vis_params, "Landsat-8")
# Map

In [21]:
#print(landsatPre.getInfo());

In [22]:
# # # # Use these bands for prediction.
# bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']

In [23]:
# #Create Pre-NDVI Map
# compositePre = landsatPre.select(bands)

# def func_ndvipre(img):
#   index = img.normalizedDifference(['B5', 'B4'])
#   clip = index.clip(geometry);#for more regions is not necessary
#   return clip.rename('Ndvi').copyProperties(img,['system:time_start','system:time_end'])

# ndviPre = compositePre.map(func_ndvipre);

In [24]:
# # maximum value composite for Before image(calculate maximum value of ndvi in every pixel)
# mvcPre = ndviPre.max();

In [26]:
# Map.addLayer(mvcPre, {min: -1, max: 1, 'palette': ['white', 'red']}, 'NDVI Before Event');
# Map

### <a name="ex3"></a> Challenge

<div class="alert alert-block alert-danger">

- Need to also calculate NDVI for post image and then map the changes.
    
    
</div> 